In [2]:
# need to read dataset from huggingface

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import time
import random
import math
import copy
import json
import re

from datasets import load_dataset


# huggingface datasets
# ! pip install datasets

In [3]:
def data_loader(index):
    url = 'shahidul034/text_summarization_dataset'
    url = url + str(index)
    dataset = load_dataset(url)

    path = 'text_summarization_dataset' + str(index) + '.csv'
    dataset['train'].to_csv(path)
    df = pd.read_csv(path)
    



    

In [ ]:
# initialize dataframe
df = pd.DataFrame(columns=['title', 'content'])
for i in range(1, 10):
    data_loader(i)
    df
# index = 2

# dataset = load_dataset(url)



In [5]:
df = pd.read_csv('text_summarization_dataset8.csv')
df

,title,content
0,বেনাপোল বন্দর পরিদর্শনে বিশ্ব ব্যাংকের প্রতিনি...,বেনাপোলের অবকাঠামোগত উন্নয়নসহ আমদানি-রপ্তানি ব...
1,খুলনায় ট্রাক-প্রাইভেটকার সংঘর্ষে ৫ জনের মৃত্যু,খুলনার রূপসা বাইপাস সড়কে এক পথচারীকে বাঁচাতে গ...
2,খুলনায় ট্রাক-প্রাইভেটকার সংঘর্ষে ৫ জনের মৃত্যু,নিহতদের নাম পরিচয় জানা যায়নি।লবণচরা থানার ভার...
3,ঈশ্বরদীতে যুবলীগ নেতা ছুরিকাহত,পাবনার ঈশ্বরদী শহরের পশ্চিম টেংরী কাচারীপাড়ায় ...
4,ঈশ্বরদীতে যুবলীগ নেতা ছুরিকাহত,রবিবার বিকেলে পৌর যুবলীগের সহ-সভাপতি স্বপনসহ ...
...,...,...
101740,খুলনাসহ ২১ জেলায় স্বাভাবিক হচ্ছে বিদ্যুৎ,এদিকে দুপুর ২টা ৩৩ মিনিটে খুলনা ও বরিশালের বে...
101741,রামগঞ্জে স্বেচ্ছাসেবক লীগের কমিটি গঠন,লক্ষ্মীপুরের রামগঞ্জ উপজেলার ভাটরা ইউনিয়ন স্বে...
101742,ফরিদপুরে ম্যাটস শিক্ষার্থীদের মানববন্ধন,কেন্দ্রীয় কর্মসূচির অংশ হিসেবে উচ্চ শিক্ষাসহ চ...
101743,লক্ষ্মীপুরে ছাত্রলীগের চার নেতা বহিষ্কার,আধিপত্য বিস্তারকে কেন্দ্র করে লক্ষ্মীপুরে ছাত্...


In [ ]:
url = 'shahidul034/text_summarization_dataset'
url = url + "2"
dataset = load_dataset(url)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 105252
    })
})

In [13]:
dataset['train'][2]

{'title': 'শাহজালালে বিমানের টয়লেটে মিলল ৪০ স্বর্ণের বার',
 'content': ' কাস্টমস কর্তৃপক্ষ সূত্র জানায়, কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের মহাপরিচালকের কাছে সংবাদ আসে সংযুক্ত আরব আমিরাতের আবুধাবি থেকে চট্টগ্রাম হয়ে আসা বিমান বাংলাদেশ এয়ারলাইনসের বিমানে অবৈধভাবে চোরাচালানকৃত স্বর্ণ বহন করা হচ্ছে। এর প্রেক্ষিতে কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের যুগ্ম মহাপরিচালক আরেফিন খানের নির্দেশে উপপরিচালক আহমেদুর রেজা চৌধুরীর নেতৃত্বে কাস্টমস গোয়েন্দার একটি দল ১০ নম্বর বোর্ডিং ব্রিজের সামনে অবস্থান নেয়।পরে কাস্টমস গোয়েন্দা দল বিমানের ভেতরে প্রবেশ করে এবং বিমানের টয়লেট থেকে বিশেষভাবে লুকিয়ে রাখা ৪০টি স্বর্ণবার উদ্ধার করে। যার মোট ওজন চর কেজি ৬৪০ গ্রাম। আটক স্বর্ণের বাজারমূল্য আনুমানিক তিন কোটি ২৪ লাখ ৮০ হাজার টাকা। এ বিষয়ে বিমানবন্দর থানায় মামলা দায়েরের প্রক্রিয়া চলছে।'}

In [15]:
from transformers import AutoTokenizer, AutoModel

bnbert_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
# text = "আমি বাংলায় গান গাই।"
# bnbert_tokenizer.tokenize(text)
# ['আমি', 'বাংলা', '##য', 'গান', 'গাই', '।']

text = dataset['train'][1]['content']
text
# 'শাহজালাল আন্তর্জাতিক বিমানবন্দরে বিমান বাংলাদেশ এয়ারলাইনসের একটি ফ্লাইটের টয়লেট থেকে বিশেষভাবে লুকিয়ে রাখা ৪০টি স্বর্ণের বার উদ্ধার করা হয়েছে। আজ সোমবার দুপুর সাড়ে ১২টার দিকে এই স্বর্ণ উদ্ধার করেন কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের কর্মকর্তারা।প্রতিটি বারের ওজন ১১৬ গ্রাম। মোট ওজন চার কেজি ৬৪০ গ্রাম বলে জানা গেছে'
bnbert_tokenizer.tokenize(text)

['শাহজালাল',
 'আনত',
 '##র',
 '##জাতিক',
 'বিমান',
 '##বন',
 '##দরে',
 'বিমান',
 'বাংলাদেশ',
 'এ',
 '##যার',
 '##লাইন',
 '##সের',
 'একটি',
 'ফলা',
 '##ইট',
 '##ের',
 'ট',
 '##য',
 '##লেট',
 'থেকে',
 'বিশেষভাবে',
 'লকি',
 '##যে',
 'রাখা',
 '[UNK]',
 'সব',
 '##র',
 '##ণের',
 'বার',
 'উদ',
 '##ধার',
 'করা',
 'হযেছে',
 '।',
 'আজ',
 'সে',
 '##াম',
 '##বার',
 'দপ',
 '##র',
 'সা',
 '##ডে',
 '[UNK]',
 'দিকে',
 'এই',
 'সব',
 '##র',
 '##ণ',
 'উদ',
 '##ধার',
 'করেন',
 'কাস',
 '##টম',
 '##স',
 'গে',
 '##ায',
 '##েন',
 '##দা',
 'ও',
 'তদ',
 '##ন',
 '##ত',
 'অধি',
 '##দ',
 '##পত',
 '##রে',
 '##র',
 'করম',
 '##কর',
 '##তারা',
 '।',
 'পরত',
 '##িটি',
 'বারের',
 'ওজন',
 '[UNK]',
 'গর',
 '##াম',
 '।',
 'মে',
 '##াট',
 'ওজন',
 'চার',
 'কেজি',
 '[UNK]',
 'গর',
 '##াম',
 'বলে',
 'জানা',
 'গেছে']